In [1]:
import json
import os


from datasets import load_dataset, concatenate_datasets

from config import load_config
from util.parsing import get_features
from util.windowing import make_random_window

/home/aaron/Documents/github/ipa-gpt-interpret/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get and Load Config and Datasets

In [2]:
configs = []
cfg = load_config(configs)
with open(cfg['mappings'], 'r') as fp:
    phoneme_mappings = json.load(fp)

print(phoneme_mappings)

{'mappings': {'ɒ': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 25, 27, -28], 'ɑ': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 25, 27, -28], 'ɶ': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 25, 26, -28], 'a': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 25, -28], 'æ': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 25, 26, -28], 'ʌ': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 27], 'ɔ': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 27], 'o': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 27, 28], 'ɤ': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 27, 28], 'ɘ': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 28], 'œ': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 26], 'ə': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23], 'e': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 26, 28], 'ɞ': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23], 'ø': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 26, 28], 'ɛ': [1, 5, 6, -7, 8, 12, -19, -20, -21, 23, 26], 'ɵ': [1, 5, 6, -7, 8, 12, 15, 16, -19, -20, -21, 23, 28], 'ɯ': [1, 5, 6, -7, 8, 1

In [5]:
rd = load_dataset(cfg['datasets']['russian'])
rd = rd.map(lambda x: {'language': 'rus'}, keep_in_memory=False)
pd = load_dataset(cfg['datasets']['polish'])
pd = pd.map(lambda x: {'language': 'pol'}, keep_in_memory=False)

Map: 100%|██████████| 1002/1002 [00:00<00:00, 35519.71 examples/s]


## Generate Windows

In [6]:
def preprocess(example):
    sentence = example['text']
    phoneme_sentence = example['text-phoneme']

    window_settings = cfg['windows']
    min_window_length = window_settings['min_size']
    max_window_length = window_settings['max_size']
    window_length_decay = window_settings['size_decay']

    """
    TODO
    because the phonemizer is word-level, you can use spaces to create boundaries for your windows
    by doing that you can create windows for both the phonetic and the orthographic columns.

    I'll leave it up to you as to how you want to encode the data into the dataset, but I'll tell you we need:
    1. The window start and stop characters
    2. The phonological features present in the span
    3. You need a window for both the orthographic column and the phonetic column
        * If you want to use the same window that's fine, but then the window offsets need to be word-based instead of character based.
        * You can see my implementation of the question answering datasets to see how you can use dicts in dataset columns: https://huggingface.co/datasets/iggy12345/sberquad-ipa (specifically see `answers` and `answers-phoneme`)
    """

    result = {}

    # Skip too short sentences
    if min(max_window_length, len(phoneme_sentence)) <= min_window_length:
        return {
            'windows': {'start': [], 'end': []},
            'windows-phoneme': {'start': [], 'end': []},
            'features': []
        }
    
    start_phoneme_char, end_phoneme_char = make_random_window(
        phoneme_sentence,
        max_window_length,
        min_window_length,
        window_length_decay,
        set(phoneme_mappings["mappings"].keys())
    )

    phoneme_sentence_window = phoneme_sentence[start_phoneme_char:end_phoneme_char+1]
    spaces_before = phoneme_sentence[0:start_phoneme_char].count(' ')
    spaces_within = phoneme_sentence[start_phoneme_char:end_phoneme_char+1].count(' ')

    start_word_idx = spaces_before
    words_in_window = spaces_within + 1
    end_word_idx = start_word_idx + words_in_window - 1

    words = sentence.split(' ')
    feature_results = get_features(phoneme_sentence_window, phoneme_mappings["mappings"])

    start_character_position = sum(len(w) for w in words[0:start_word_idx]) + start_word_idx
    end_character_position = sum(len(w) for w in words[start_word_idx:end_word_idx+1]) + (end_word_idx - start_word_idx) + start_character_position - 1

    result = {
        'windows': {
            'start': [start_character_position],
            'end': [end_character_position]
        },
        'windows-phoneme': {
            'start': [start_phoneme_char],
            'end': [end_phoneme_char]
        },
        'features': [feature_results]
    }
    return result

In [7]:
keep_columns = ['text', 'text-phoneme']

import time
timestamp = int(time.time())

rd_sampled = rd.map(
    preprocess,
    remove_columns=[c for c in rd['train'].column_names if c not in keep_columns],
    # cache_file_names={
    #     'train': f'C:/Users/wayne/.cache/huggingface/datasets/cache_rd_train_{timestamp}.arrow',
    #     'validation': f'C:/Users/wayne/.cache/huggingface/datasets/cache_rd_val_{timestamp}.arrow',
    #     'test': f'C:/Users/wayne/.cache/huggingface/datasets/cache_rd_test_{timestamp}.arrow'
    # },
    # load_from_cache_file=False
)

timestamp = int(time.time())

pd_sampled = pd.map(
    preprocess,
    remove_columns=[c for c in pd['train'].column_names if c not in keep_columns],
    # cache_file_names={
    #     'train': f'C:/Users/wayne/.cache/huggingface/datasets/cache_pd_train_{timestamp}.arrow',
    #     'validation': f'C:/Users/wayne/.cache/huggingface/datasets/cache_pd_val_{timestamp}.arrow',
    #     'test': f'C:/Users/wayne/.cache/huggingface/datasets/cache_pd_test_{timestamp}.arrow'
    # },
    # load_from_cache_file=False
)

Map: 100%|██████████| 1002/1002 [00:00<00:00, 11635.10 examples/s]


# Concatenate Datasets

In [8]:
combined_ds = {}
for split in rd_sampled.keys():
    combined_ds[split] = concatenate_datasets([rd_sampled[split], pd_sampled[split]])

from datasets import DatasetDict
combined_ds = DatasetDict(combined_ds)


### Check dataset

In [9]:
print("Checking first 10 examples:")
for i in range(10):
    ex = combined_ds['train'][i]
    if ex['windows']['start']:
        phon_window = ex['text-phoneme'][ex['windows-phoneme']['start'][0]:ex['windows-phoneme']['end'][0]+1]
        print(f"Example {i}: phoneme_window='{phon_window}' -> features={ex['features'][0]}")

Checking first 10 examples:
Example 0: phoneme_window='sˈup' -> features=[1, 4, 5, 6, 7, 8, 12, 15, 16, 18, 19, 21, 23, 24, 27, 28, -26, -25, -20]
Example 1: phoneme_window='a greenish' -> features=[1, 4, 5, 6, 7, 8, 10, 11, 12, 13, 18, 19, 21, 23, 24, 25, 26, 28, -27, -20]
Example 2: phoneme_window='ersʲikʌvyj' -> features=[1, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 21, 23, 24, 26, 27, 28, -25, -20]
Example 3: phoneme_window='prʲ' -> features=[4, -27, -26, -25, -24, -28, 5, -21, -20, 6, 8, 15, 10, 12, 18, 19, -7]
Example 4: phoneme_window='ɑ d' -> features=[1, -28, 5, 6, 4, 8, -26, -24, -21, 12, -20, 18, 19, 23, -7, 27, 25]
Example 5: phoneme_window='svʲˈɑʑɪ' -> features=[1, 4, 5, 6, 7, 8, 12, 15, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, -28]
Example 6: phoneme_window='onʲink' -> features=[1, 4, 5, 6, 8, 11, 12, 15, 16, 18, 19, 23, 24, 26, 27, 28, -25, -21, -20, -7]
Example 7: phoneme_window='ɛ mˈɑɭ' -> features=[1, 4, 5, 6, 8, 11, 12, 15, 18, 22, 23, 25, 26, 27, -28, -24, -21, -20,

## Push to Huggingface

In [10]:
# TODO if you get here, you're done! I'll handle pushing the dataset to the cloud

target_dataset_name = 'rus-pol-edge-probing-phono-feats'
username = 'iggy12345'

In [11]:
combined_ds.push_to_hub(f'{username}/{target_dataset_name}')

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  7.51ba/s]
Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (0 / 1):  98%|█████████▊| 18.7MB / 19.0MB, 1.36MB/s  
Processing Files (1 / 1): 100%|██████████| 19.0MB / 19.0MB, 1.14MB/s  
Processing Files (1 / 1): 100%|██████████| 19.0MB / 19.0MB,  909kB/s  
New Data Upload: 100%|██████████|  909kB /  909kB,  909kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 27.67ba/s]
Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|██████████| 4.70MB / 4.70MB, 1.64MB/s  
Processing Files (1 / 1): 100%|██████████| 4.70MB / 4.70MB,  235kB/s  
New Data Upload: 100%|██████████|  328kB /  328kB,  235kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 29.51ba/s]
Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|██████████| 4.71MB / 4.71MB, 1.72MB/s  
Pro

CommitInfo(commit_url='https://huggingface.co/datasets/iggy12345/rus-pol-edge-probing-phono-feats/commit/bba430c53e8059599f75e4b8e5e8c17b70331f38', commit_message='Upload dataset', commit_description='', oid='bba430c53e8059599f75e4b8e5e8c17b70331f38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/iggy12345/rus-pol-edge-probing-phono-feats', endpoint='https://huggingface.co', repo_type='dataset', repo_id='iggy12345/rus-pol-edge-probing-phono-feats'), pr_revision=None, pr_num=None)